In [1]:
import pandas as pd
import oracledb as odb
import sqlalchemy as sqa

In [10]:
odb.init_oracle_client()

data_inicial = ""
data_final = ""

print("Data_Inicial: ")
data_inicial = input()

print("Data_Final: ")
data_final = input()

print(data_inicial)
print(data_final)

Data_Inicial: 
Data_Final: 
01/08/2024
31/01/2025


In [ ]:
# connect to oracle database
connection=odb.connect(
     user='rrgraciano',
     password='rrgraciano',
     dsn='10.10.8.60/tasyprd',
     #config_dir=wallet_location,
     #wallet_location=wallet_location,
     #wallet_password=wallet_pw)
)
# create engine
engine = sqa.create_engine('oracle+oracledb://', creator=lambda: connection)
# Get the data into a DataFrame
query = f"""
SELECT *
FROM (
    -- Consulta Base (Unindo as duas consultas anteriores)
    SELECT
    	CASE
    		WHEN TRIM(a.DS_CURTA) = 'AVALIAÇÃO DE INJECAO' THEN 'AVALIACAO DE INJECAO'
    		WHEN TRIM(a.DS_CURTA) = 'AVALIAÇÃO DE INJEÇÃO' THEN 'AVALIACAO DE INJECAO'
    		WHEN TRIM(a.DS_CURTA) = 'REFRACAO &#x2013; EXTERNA' THEN 'EXTERNA - REFRACAO'
    		WHEN TRIM(a.DS_CURTA) = 'EXTERNA - REFRAÇÃO' THEN 'EXTERNA - REFRACAO'
    		WHEN TRIM(a.DS_CURTA) = 'REFRAÇÃO' THEN 'REFRACAO'
    		WHEN TRIM(a.DS_CURTA) ='CATARATA ENTRADA - EXTERNA' THEN 'EXTERNA - CATARATA'	
			WHEN TRIM(a.DS_CURTA) ='EXTERNA - PLÁSTICA' THEN 'EXTERNA - PLASTICA'
			WHEN TRIM(a.DS_CURTA) ='EXTERNA - RETINA CIRÚRGICA' THEN 'EXTERNA - RETINA CIRURGICA'
			WHEN TRIM(a.DS_CURTA) ='RETINA CIRÚRGICA' THEN 'RETINA CIRURGICA'
			WHEN TRIM(a.DS_CURTA) ='GLAUCOMA INCLUSÃO/REAVALIAÇÃO' THEN 'GLAUCOMA INCLUSAO/REAVALIACAO'
			WHEN TRIM(a.DS_CURTA) ='YAG LASER &#x2013; EXTERNA' THEN 'EXTERNA - YAG LASER'
			WHEN TRIM(a.DS_CURTA) ='TRIAGEM DE NEUROFTALMOLOGIA' THEN 'TRIAGEM NEUROFTALMOLOGIA'
			WHEN TRIM(a.DS_CURTA) ='OFTALMOPEDIATRIA - EXTERNA' THEN 'EXTERNA - OFTALMOPEDIATRIA'
			WHEN TRIM(a.DS_CURTA) ='PROJETO MIGUILIM - EXTERNA' THEN 'EXTERNA - PROJETO MIGUILIM'
			WHEN TRIM(a.DS_CURTA) ='PEQUENOS PROCEDIMENTOS-EXTERNA' THEN 'EXTERNA - PEQUENOS PROC'
			WHEN TRIM(a.DS_CURTA) ='OFTALMOPEDIATRIA - EXTERNA' THEN 'EXTERNA - OFTALMOPEDIATRIA'
			WHEN TRIM(a.DS_CURTA) ='NEUROFTALMOLOGIA - EXTERNA' THEN 'EXTERNA - NEUROFTALMOLOGIA'
			WHEN TRIM(a.DS_CURTA) ='FUNDOSCOPIA DIABETICA -EXTERNA' THEN 'EXTERNA - FUNDOSCOPIA'
			WHEN TRIM(a.DS_CURTA) ='AVALIACAO DE INJECAO - EXTERNA' THEN 'EXTERNA - AVALIAÇÃO DE INJECAO'
			WHEN TRIM(a.DS_CURTA) ='FOTOCOAGULAÇÃO' THEN 'FOTOCOAGULACAO'
			WHEN TRIM(a.DS_CURTA) ='POS OP TRANSPLANTE DE CORNEA' THEN 'POS-OPERATORIO'
			WHEN TRIM(a.DS_CURTA) ='POS-OP PEQUENOS PROCEDIMENTOS' THEN 'POS-OPERATORIO'
			WHEN TRIM(a.DS_CURTA) ='PÓS OP - GLAUCOMA CIRURGICO' THEN 'POS-OPERATORIO'
			WHEN TRIM(a.DS_CURTA) ='PÓS OP - PLASTICA' THEN 'POS-OPERATORIO'
			WHEN TRIM(a.DS_CURTA) ='PÓS OP - PTERIGIO' THEN 'POS-OPERATORIO'
			WHEN TRIM(a.DS_CURTA) ='PÓS OP CATARATA' THEN 'POS-OPERATORIO'
			WHEN TRIM(a.DS_CURTA) ='PLASTICA POS-OPERATORIO' THEN 'POS-OPERATORIO'
			WHEN TRIM(a.DS_CURTA) ='GLAUCOMA CIRÚRGICO PÓS' THEN 'POS-OPERATORIO'
			ELSE TRIM(a.DS_CURTA)
    	END AS ESPECIALIDADE,
        TO_CHAR(DT_AGENDA, 'IW-YYYY') AS DATAS,  -- Ajustado para semana ISO
        COUNT(ac.CD_PESSOA_FISICA) AS AGENDADOS
    FROM tasy.AGENDA_CONSULTA ac 
    INNER JOIN TASY.AGENDA a ON ac.CD_AGENDA = a.CD_AGENDA 
    WHERE DT_AGENDA BETWEEN TO_DATE('{data_inicial}', 'DD/MM/YYYY') AND TO_DATE('{data_final}', 'DD/MM/YYYY')
    AND IE_STATUS_AGENDA NOT IN ('C','B','I')
    AND a.CD_ESTABELECIMENTO = 8
    GROUP BY 
        TRIM(a.DS_CURTA),
        TO_CHAR(DT_AGENDA, 'IW-YYYY')

    UNION 

    SELECT 
        TRIM(a.DS_AGENDA) AS ESPECIALIDADE,
        TO_CHAR(DT_AGENDA, 'IW-YYYY') AS DATAS,
        COUNT(ap.CD_PESSOA_FISICA) AS AGENDADOS
    FROM tasy.AGENDA_PACIENTE ap 
    INNER JOIN TASY.AGENDA a ON ap.CD_AGENDA = a.CD_AGENDA
    WHERE DT_AGENDA BETWEEN TO_DATE('{data_inicial}', 'DD/MM/YYYY') AND TO_DATE('{data_final}', 'DD/MM/YYYY')
    AND IE_STATUS_AGENDA NOT IN ('C','B','I')
    AND a.CD_ESTABELECIMENTO = 8
    GROUP BY 
        TRIM(a.DS_AGENDA),
        TO_CHAR(DT_AGENDA, 'IW-YYYY')
) A
INNER JOIN
(SELECT 
	to_char(dt_entrada,'IW-YYYY') AS DATAS,  -- Ajustado para semana ISO
	SUM(vl_procedimento) AS FATURADO
FROM TASY.EIS_VALORES_ATEND_PACIENTE_V
WHERE dt_entrada BETWEEN TO_DATE('{data_inicial}', 'DD/MM/YYYY') AND TO_DATE('{data_final}', 'DD/MM/YYYY')
AND cd_estabelecimento = 8 
GROUP BY to_char(dt_entrada,'IW-YYYY')) B ON A.DATAS = B.DATAS
PIVOT (
    MAX(AGENDADOS)
    FOR ESPECIALIDADE IN (    	
        '1 - SALA VERDE' AS "1 - SALA VERDE",
		'2 - SALA AZUL' AS "2 - SALA AZUL",
		'ANGIOGRAFIA FLUORESCENTE' AS "ANGIOGRAFIA FLUORESCENTE",
		'AVALIACAO DE INJECAO' AS "AVALIACAO DE INJECAO",
		'BLEFARO' AS "BLEFARO",
		'CATARATA ENTRADA' AS "CATARATA ENTRADA",
		'CATARATA RETORNO' AS "CATARATA RETORNO",
		'CORNEA' AS "CORNEA",
		'CURVA DE PRESSAO' AS "CURVA DE PRESSAO",
		'ECO B' AS "ECO B",
		'ESTRABISMO' AS "ESTRABISMO",
		'EXTERNA - AVALIAÇÃO DE INJECAO' AS "EXTERNA - AVALIAÇÃO DE INJECAO",
		'EXTERNA - BLEFARO' AS "EXTERNA - BLEFARO",
		'EXTERNA - CATARATA' AS "EXTERNA - CATARATA",
		'EXTERNA - ECO B' AS "EXTERNA - ECO B",
		'EXTERNA - ESTRABISMO' AS "EXTERNA - ESTRABISMO",
		'EXTERNA - FOTOCOAGULACAO' AS "EXTERNA - FOTOCOAGULACAO",
		'EXTERNA - FUNDOSCOPIA' AS "EXTERNA - FUNDOSCOPIA",
		'EXTERNA - GLAUCOMA INCLUSAO' AS "EXTERNA - GLAUCOMA INCLUSAO",
		'EXTERNA - NEUROFTALMOLOGIA' AS "EXTERNA - NEUROFTALMOLOGIA",
		'EXTERNA - OFTALMOPEDIATRIA' AS "EXTERNA - OFTALMOPEDIATRIA",
		'EXTERNA - PEQUENOS PROC' AS "EXTERNA - PEQUENOS PROC",
		'EXTERNA - PLASTICA' AS "EXTERNA - PLASTICA",
		'EXTERNA - PROJETO MIGUILIM' AS "EXTERNA - PROJETO MIGUILIM",
		'EXTERNA - PTERIGIO' AS "EXTERNA - PTERIGIO",
		'EXTERNA - REFRACAO' AS "EXTERNA - REFRACAO",
		'EXTERNA - RETINA' AS "EXTERNA - RETINA",
		'EXTERNA - RETINA CIRURGICA' AS "EXTERNA - RETINA CIRURGICA",
		'EXTERNA - RETINA CLINICA' AS "EXTERNA - RETINA CLINICA",
		'EXTERNA - TESTE DE LENTE' AS "EXTERNA - TESTE DE LENTE",
		'EXTERNA - YAG LASER' AS "EXTERNA - YAG LASER",
		'FOTOCOAGULACAO' AS "FOTOCOAGULACAO",
		'GLAUCOMA ACOMPANHAMENTO' AS "GLAUCOMA ACOMPANHAMENTO",
		'GLAUCOMA CIRURGICO' AS "GLAUCOMA CIRURGICO",
		'GLAUCOMA INCLUSAO/REAVALIACAO' AS "GLAUCOMA INCLUSAO/REAVALIACAO",
		'NEUROFTALMOLOGIA' AS "NEUROFTALMOLOGIA",
		'OFTALMOPEDIATRIA' AS "OFTALMOPEDIATRIA",
		'PEQUENOS PROCEDIMENTOS' AS "PEQUENOS PROCEDIMENTOS",
		'PLASTICA' AS "PLASTICA",
		'REFRACAO' AS "REFRACAO",
		'REFRACAO RELATORIO' AS "REFRACAO RELATORIO",
		'RETINA CIRURGICA' AS "RETINA CIRURGICA",
		'RETINA CLINICA' AS "RETINA CLINICA",
		'RETINOGRAFIA COLORIDA' AS "RETINOGRAFIA COLORIDA",
		'RISCO CIRURGICO' AS "RISCO CIRURGICO",
		'SALA PEQUENOS PROCEDIMENTOS' AS "SALA PEQUENOS PROCEDIMENTOS",
		'TESTE DE LENTE' AS "TESTE DE LENTE",
		'TRANSPLANTE DE CORNEA' AS "TRANSPLANTE DE CORNEA",
		'TRIAGEM NEUROFTALMOLOGIA' AS "TRIAGEM NEUROFTALMOLOGIA",
		'UVEITES' AS "UVEITES",
		'YAG LASER' AS "YAG LASER"
    )
)
"""
df = pd.read_sql(query, engine)


In [12]:
df = df.drop(columns=["qcsj_c000000001000001"])
df = df.rename(columns={"qcsj_c000000001000000": "data"})
df = df.dropna(axis=1, how='all')
df = df.drop(columns=["data"])
df = df.fillna(0)
df = df.drop(0, axis=0).reset_index(drop=True)
df

,faturado,1 - SALA VERDE,2 - SALA AZUL,ANGIOGRAFIA FLUORESCENTE,AVALIACAO DE INJECAO,blefaro,CATARATA ENTRADA,CATARATA RETORNO,cornea,CURVA DE PRESSAO,...,RETINA CIRURGICA,RETINA CLINICA,RETINOGRAFIA COLORIDA,RISCO CIRURGICO,SALA PEQUENOS PROCEDIMENTOS,TESTE DE LENTE,TRANSPLANTE DE CORNEA,TRIAGEM NEUROFTALMOLOGIA,uveites,YAG LASER
0,207915.64,33,0,19.0,33,7.0,25.0,1.0,29,4.0,...,0.0,69,0.0,0.0,160,0.0,0.0,2.0,0.0,32.0
1,986328.42,72,301,43.0,156,60.0,239.0,51.0,110,7.0,...,154.0,405,3.0,75.0,481,0.0,0.0,4.0,27.0,70.0
2,960800.24,51,280,41.0,141,61.0,283.0,59.0,101,5.0,...,164.0,480,2.0,74.0,287,36.0,1.0,7.0,22.0,71.0
3,930698.02,79,367,55.0,141,70.0,354.0,51.0,92,2.0,...,140.0,442,2.0,75.0,192,34.0,0.0,9.0,21.0,72.0
4,891603.32,52,303,32.0,104,76.0,296.0,57.0,102,0.0,...,86.0,413,6.0,0.0,205,28.0,0.0,0.0,28.0,45.0
5,309524.65,7,38,0.0,30,19.0,82.0,47.0,36,5.0,...,72.0,125,0.0,76.0,69,11.0,0.0,0.0,0.0,32.0
6,869325.38,35,71,13.0,113,40.0,225.0,46.0,86,10.0,...,112.0,367,7.0,73.0,173,24.0,0.0,10.0,16.0,58.0
7,619396.35,53,45,14.0,100,38.0,115.0,0.0,89,7.0,...,108.0,335,3.0,66.0,105,9.0,0.0,5.0,19.0,63.0
8,851270.35,51,244,13.0,94,58.0,234.0,46.0,89,5.0,...,122.0,444,5.0,65.0,204,24.0,0.0,5.0,21.0,72.0
9,901988.85,166,313,12.0,118,56.0,290.0,42.0,131,6.0,...,161.0,435,4.0,65.0,229,20.0,0.0,5.0,20.0,62.0


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as ln
from sklearn.metrics import mean_squared_error

In [14]:
X = df.drop(columns=["faturado"])
y = df["faturado"]

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
model = ln()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test,y_pred)
print(f"Erro quadrático médio: {mse}")

Erro quadrático médio: 15501704532.93714


In [17]:
y_pred

array([860574.4635, 829257.1902, 858314.0617, 789949.4553, 487475.0808,
       763191.6037])

In [18]:
y_test

8     851270.35
13    812217.22
9     901988.85
21    857148.57
0     207915.64
11    852950.23
Name: faturado, dtype: float64

In [19]:
import numpy as np
np.sqrt(mse)

124505.8413607054

In [20]:
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,  # Pegando os nomes das colunas
    'Importance': model.feature_importances_  # Pegando as importâncias
})

# Ordenando as features pela importância
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Exibindo o resultado
print(feature_importances)






                           Feature  Importance
5                 CATARATA ENTRADA    0.190017
4                          blefaro    0.128954
42                  RETINA CLINICA    0.081205
1                    2 - SALA AZUL    0.052353
41                RETINA CIRURGICA    0.051233
21         EXTERNA - PEQUENOS PROC    0.047780
33              GLAUCOMA CIRURGICO    0.034974
34   GLAUCOMA INCLUSAO/REAVALIACAO    0.033688
31                  fotocoagulacao    0.029256
32         GLAUCOMA ACOMPANHAMENTO    0.027312
28        EXTERNA - RETINA CLINICA    0.024959
0                   1 - SALA VERDE    0.023001
36                oftalmopediatria    0.022498
50                       YAG LASER    0.021016
39                        refracao    0.020591
45     SALA PEQUENOS PROCEDIMENTOS    0.019566
46                  TESTE DE LENTE    0.017883
44                 RISCO CIRURGICO    0.017687
37          PEQUENOS PROCEDIMENTOS    0.014491
43           RETINOGRAFIA COLORIDA    0.014255
38           